In [1]:
import time
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
import pytorch3d.ops
from plyfile import PlyData, PlyElement
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from argparse import ArgumentParser, Namespace
import cv2

from arguments import ModelParams, PipelineParams
from scene import Scene, GaussianModel, FeatureGaussianModel
from gaussian_renderer import render, render_contrastive_feature

#import utils.contrastive_decoder_utils
from utils.sh_utils import SH2RGB

def get_combined_args(parser : ArgumentParser, model_path, target_cfg_file = None):
    cmdlne_string = ['--model_path', model_path]
    cfgfile_string = "Namespace()"
    args_cmdline = parser.parse_args(cmdlne_string)
    
    if target_cfg_file is None:
        if args_cmdline.target == 'seg':
            target_cfg_file = "seg_cfg_args"
        elif args_cmdline.target == 'scene' or args_cmdline.target == 'xyz':
            target_cfg_file = "cfg_args"
        elif args_cmdline.target == 'feature' or args_cmdline.target == 'coarse_seg_everything' or args_cmdline.target == 'contrastive_feature' :
            target_cfg_file = "feature_cfg_args"

    try:
        cfgfilepath = os.path.join(model_path, target_cfg_file)
        print("Looking for config file in", cfgfilepath)
        with open(cfgfilepath) as cfg_file:
            print("Config file found: {}".format(cfgfilepath))
            cfgfile_string = cfg_file.read()
    except TypeError:
        print("Config file found: {}".format(cfgfilepath))
        pass
    args_cfgfile = eval(cfgfile_string)

    merged_dict = vars(args_cfgfile).copy()
    for k,v in vars(args_cmdline).items():
        if v != None:
            merged_dict[k] = v

    return Namespace(**merged_dict)

In [2]:
import os
FEATURE_DIM = 32 # fixed

# MODEL_PATH = './output/lerf-fruit_aisle/'
MODEL_PATH = './output/lund_1024' # 30000

FEATURE_GAUSSIAN_ITERATION = 10000

SCALE_GATE_PATH = os.path.join(MODEL_PATH, f'point_cloud/iteration_{str(FEATURE_GAUSSIAN_ITERATION)}/scale_gate.pt')

FEATURE_PCD_PATH = os.path.join(MODEL_PATH, f'point_cloud/iteration_{str(FEATURE_GAUSSIAN_ITERATION)}/contrastive_feature_point_cloud.ply')
SCENE_PCD_PATH = os.path.join(MODEL_PATH, f'point_cloud/iteration_{str(FEATURE_GAUSSIAN_ITERATION)}/scene_point_cloud.ply')

In [3]:
scale_gate = torch.nn.Sequential(
    torch.nn.Linear(1, 32, bias=True),
    torch.nn.Sigmoid()
)

scale_gate.load_state_dict(torch.load(SCALE_GATE_PATH))
scale_gate = scale_gate.cuda()

parser = ArgumentParser(description="Testing script parameters")
model = ModelParams(parser, sentinel=True)
pipeline = PipelineParams(parser)
parser.add_argument('--target', default='scene', type=str)

args = get_combined_args(parser, MODEL_PATH)

dataset = model.extract(args)

# If use language-driven segmentation, load clip feature and original masks
dataset.need_features = True

# To obtain mask scales
dataset.need_masks = True

scene_gaussians = GaussianModel(dataset.sh_degree)

feature_gaussians = FeatureGaussianModel(FEATURE_DIM)
scene = Scene(dataset, scene_gaussians, feature_gaussians, load_iteration=-1, feature_load_iteration=FEATURE_GAUSSIAN_ITERATION, shuffle=False, mode='eval', target='contrastive_feature')


Looking for config file in ./output/lund_1024/cfg_args
Config file found: ./output/lund_1024/cfg_args
Loading trained model at iteration 30000, 10000
Allow Camera Principle Point Shift: False
Reading camera 1196/1196
✅ Loaded 1196 cameras for this GPU (start_idx=0, end_idx=None)
Loading Training Cameras
Loading Test Cameras


In [4]:
import importlib
import clip_utils
importlib.reload(clip_utils)
from clip_utils import get_scores_with_template
from clip_utils.clip_utils import load_clip
clip_model = load_clip()
clip_model.eval()
1

Embedding dimension 512


1

In [5]:
from sklearn.preprocessing import QuantileTransformer
# Borrowed from GARField, but modified
def get_quantile_func(scales: torch.Tensor, distribution="normal"):
    """
    Use 3D scale statistics to normalize scales -- use quantile transformer.
    """
    scales = scales.flatten()

    scales = scales.detach().cpu().numpy()
    print(scales.max(), '?')

    # Calculate quantile transformer
    quantile_transformer = QuantileTransformer(output_distribution=distribution)
    quantile_transformer = quantile_transformer.fit(scales.reshape(-1, 1))

    
    def quantile_transformer_func(scales):
        scales_shape = scales.shape

        scales = scales.reshape(-1,1)
        
        return torch.Tensor(
            quantile_transformer.transform(scales.detach().cpu().numpy())
        ).to(scales.device).reshape(scales_shape)

    return quantile_transformer_func, quantile_transformer
    
all_scales = []
all_scales = []
for cam in scene.getTrainCameras():
    scale_path = os.path.join(dataset.source_path, 'mask_scales', cam.image_name + '.pt')
    scales = torch.load(scale_path)
    all_scales.append(scales)

all_scales = torch.cat(all_scales)

upper_bound_scale = all_scales.max().item()
# upper_bound_scale = np.percentile(all_scales.detach().cpu().numpy(), 75)

# all_scales = []
# for cam in scene.getTrainCameras():
#     cam.mask_scales = torch.clamp(cam.mask_scales, 0, upper_bound_scale).detach()
#     all_scales.append(cam.mask_scales)
# all_scales = torch.cat(all_scales)

# quantile transformer
q_trans, q_trans_ = get_quantile_func(all_scales, 'uniform')




18.245272 ?


In [6]:
from copy import deepcopy

seg_features = []
clip_features = []
scales = []
mask_identifiers = []
camera_id_mask_id = []

bg_color = [0 for _ in range(FEATURE_DIM)]
background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

for i, view in enumerate(scene.getTrainCameras()):
    torch.cuda.empty_cache()
    clip_features.append(view.original_features)

    tmp_view = deepcopy(view)
    tmp_view.feature_height, tmp_view.feature_width = view.original_image.shape[-2:]

    rendered_feature = render_contrastive_feature(
        tmp_view,
        feature_gaussians,
        pipeline.extract(args),
        background,
        norm_point_features=True
    )['render']

    feature_h, feature_w = rendered_feature.shape[-2:]
    rendered_feature = torch.nn.functional.interpolate(
        rendered_feature.unsqueeze(0), (feature_h // 4, feature_w // 4), mode='bilinear'
    ).squeeze()

    sam_masks = torch.load(view.original_masks).cuda().unsqueeze(1)
    sam_masks = torch.nn.functional.interpolate(sam_masks.float(), (feature_h // 4, feature_w // 4), mode='bilinear')
    sam_masks = torch.conv2d(
        sam_masks.float().cpu(),
        torch.full((3, 3), 1.0).view(1, 1, 3, 3).cpu(),
        padding=1
    )
    sam_masks = sam_masks >= 2
    sam_masks = sam_masks.cuda()

    mask_scales = torch.load(view.mask_scales).cuda().unsqueeze(-1)
    mask_scales = q_trans(mask_scales)
    scale_gates = scale_gate(mask_scales)

    scale_conditioned_feature = rendered_feature.unsqueeze(0) * scale_gates.unsqueeze(-1).unsqueeze(-1)
    scale_conditioned_feature = torch.nn.functional.normalize(scale_conditioned_feature, dim=1, p=2)

    mask_features = (sam_masks * scale_conditioned_feature).sum(dim=-1).sum(dim=-1) / (
        sam_masks.sum(dim=-1).sum(dim=-1) + 1e-9
    )
    mask_features = torch.nn.functional.normalize(mask_features, dim=-1, p=2)

    mask_ident = torch.einsum('nc,nc->n', mask_features, mask_features) > 0.5  # dummy consistency
    mask_identifiers.append(mask_ident.cpu())
    seg_features.append(mask_features)
    scales.append(mask_scales)

    for j in range(len(mask_features)):
        camera_id_mask_id.append((i, j))


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.22 GiB (GPU 0; 31.73 GiB total capacity; 30.13 GiB already allocated; 294.94 MiB free; 31.04 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Flatten all features from all cameras
flattened_mask_features = torch.cat(seg_features, dim=0)
clip_features = [torch.load(path).cuda() for path in clip_features]
flattened_clip_features = torch.cat(clip_features, dim=0)
flattened_clip_features = torch.nn.functional.normalize(flattened_clip_features.float(), dim=-1, p=2)

flattened_scales = torch.cat(scales, dim=0)
flattened_mask_identifiers = torch.cat(mask_identifiers, dim=0).to(torch.float16).cuda()


In [ ]:
clip_model = load_clip()
clip_model.eval()

prompt = "house"
scores = get_scores_with_template(clip_model, flattened_clip_features.cuda(), prompt).squeeze()

best_idx = scores.argmax()
query_feature = flattened_mask_features[best_idx]
query_scale = flattened_scales[best_idx].item()

similarities = get_similarity_map(
    feature_gaussians.get_point_features,
    query_scale,
    scale_gate,
    query_feature,
    q_trans
)

scene_gaussians.roll_back()
scene_gaussians.segment(similarities > 0.45)

background = torch.tensor([1.0] * FEATURE_DIM, dtype=torch.float32, device='cuda')
rendered = render(scene.getTrainCameras()[camera_index], scene_gaussians, pipeline.extract(args), background)['render']

plt.imshow(rendered.permute(1, 2, 0).detach().cpu())
plt.axis('off')
plt.title(f"3D Segmentation via Prompt: '{prompt}'")
plt.show()


In [ ]:
from copy import deepcopy
camera_index = 17  # Camera to extract 2D patch from
query_indices = [(100, 800)]  # Pixel positions (H, W)
threshold = 0.75
# --- 1. Extract 2D feature map from camera ---
view = scene.getTrainCameras()[camera_index]
tmp_view = deepcopy(view)
tmp_view.feature_height, tmp_view.feature_width = view.original_image.shape[-2:]

bg_color = [0 for _ in range(FEATURE_DIM)]
background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

render_pkg = render_contrastive_feature(
    tmp_view,
    feature_gaussians,
    pipeline.extract(args),
    background,
    norm_point_features=True
)
rendered_feature = render_pkg['render']

In [ ]:
# Downsample to match scale with SAM or visual inspection
feature_h, feature_w = rendered_feature.shape[-2:]
rendered_feature = torch.nn.functional.interpolate(
    rendered_feature.unsqueeze(0), (feature_h // 4, feature_w // 4), mode='bilinear'
).squeeze(0)

In [ ]:
# --- 2. Normalize and extract query features ---
normed_features = torch.nn.functional.normalize(rendered_feature, dim=0, p=2).permute(1, 2, 0)  # [H, W, C]
similarity_list = []

for qi in query_indices:
    y = int(qi[0] / view.image_height * normed_features.shape[0])
    x = int(qi[1] / view.image_width * normed_features.shape[1])
    qf = normed_features[y, x]  # [C]
    sim = torch.einsum('C,HWC->HW', qf.cuda(), normed_features)
    similarity_list.append(sim)

final_similarity_2d = torch.stack(similarity_list, dim=0).max(dim=0)[0]  # [H, W]

In [ ]:
# --- 3. Visualize 2D similarity and threshold mask ---
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(final_similarity_2d.detach().cpu(), cmap='hot')
plt.title("2D Similarity Heatmap")
plt.subplot(1, 2, 2)
plt.imshow((final_similarity_2d > threshold).detach().cpu())
plt.title("Thresholded 2D Mask")
plt.show()

In [ ]:
visible_mask = render_pkg['visibility_filter']  # shape: [N_total], bool
mask_flat = (final_similarity_2d > threshold).float().flatten()
print("visible_mask shape:", visible_mask.shape)
print("Number of visible points:", visible_mask.sum().item())
print("final_similarity_2d shape:", final_similarity_2d.shape)
print("mask_flat shape:", mask_flat.shape)

# Safe fix: crop mask_flat to match visible count
mask_flat = mask_flat[:visible_mask.sum()]

full_mask = torch.zeros_like(visible_mask, dtype=torch.bool)
full_mask[visible_mask] = mask_flat.bool()

print(f"✅ Projected {full_mask.sum().item()} 2D-selected points into 3D")


In [ ]:
cameras = scene.getTrainCameras()
rendered = render(cameras[17], scene_gaussians, pipeline.extract(args), background)['render']
plt.imshow(rendered.permute(1, 2, 0).detach().cpu())


In [ ]:
# --- 5. Apply segmentation ---
try:
    scene_gaussians.roll_back()
except:
    pass
scene_gaussians.segment(full_mask)

In [ ]:
# --- 6. Render result ---
background = torch.tensor([1.0]*FEATURE_DIM, dtype=torch.float32, device="cuda")
rendered = render(cameras[camera_index], scene_gaussians, pipeline.extract(args), background)['render']

plt.imshow(rendered.permute(1, 2, 0).detach().cpu())
plt.axis('off')
plt.title("3D Segmentation via 2D Patch Query")
plt.show()

In [ ]:
# --- 7. Optional: Save for later rendering ---
torch.save(full_mask.cpu(), './segmentation_res/precomputed_mask.pt')
print("✅ Saved precomputed_mask.pt for render.py")

In [ ]:
from typing import Callable

def get_similarity_map(
    point_features: torch.Tensor,
    scale: float,
    scale_gate: Callable,
    clip_query_feature: torch.Tensor,
    q_trans: Callable[[torch.Tensor], torch.Tensor]
):
    scale = torch.tensor([scale], device='cuda')
    scale = q_trans(scale)
    gates = scale_gate(scale).squeeze()
    
    scale_conditioned = point_features * gates.unsqueeze(0)
    normed = torch.nn.functional.normalize(scale_conditioned, dim=-1, p=2)
    similarities = torch.einsum('C,NC->N', clip_query_feature, normed)
    
    return similarities


In [ ]:
# Step 1: Extract
point_features = feature_gaussians.get_point_features
point_features = torch.nn.functional.normalize(point_features, dim=-1, p=2)

# Step 2: Pick a CLIP prompt
from utils.clip_utils import get_scores_with_template
prompt = "house"
scores = get_scores_with_template(clip_model, point_features, prompt).squeeze()

# Step 3: Use highest scoring point as query
top_index = scores.argmax()
clip_query_feature = point_features[top_index]
scale = 0.5  # Or adapt from your clusters
similarities = get_similarity_map(point_features, scale, scale_gate, clip_query_feature, q_trans)

# Step 4: Threshold and segment
threshold = 0.45
scene_gaussians.roll_back()
scene_gaussians.segment(similarities > threshold)


In [ ]:
background = torch.tensor([1.0] * FEATURE_DIM, dtype=torch.float32, device='cuda')
rendered = render(scene.getTrainCameras()[17], scene_gaussians, pipeline.extract(args), background)['render']
plt.imshow(rendered.permute(1, 2, 0).detach().cpu())
plt.axis('off')
plt.title("Language-Driven Segmentation")
plt.show()
